## 08 测试和调试
本节介绍与测试、日志和调试有关的基本主题。


### 01 测试
#### **多测试，少调试（Testing Rocks，Debugging Sucks）**
Python的动态性质使得测试对大多数程序而言至关重要。编译器不会发现你的bug，发现bug的唯一方式是运行代码，并确保尝试了所有的特性。
#### **断言（Assertions）**
`assert` 语句用于程序的内部检查。如果表达式不为真，则会触发 `AssertionError` 异常。

`assert `语句语法：
```
assert <expression> [, 'Diagnostic message']
```
示例：
```
assert isinstance(10, int), 'Expected int'
```
`assert` 语句不应用于检查用户的输入（例如，在网页表单输入的数据）。 `assert` 语句旨在用于内部检查或者用于不变量（invariant，始终为 True 的条件）。
#### **契约式编程**
契约式编程（contract programming）也称为契约式设计（Design By Contract），自由使用断言是一种软件设计方法。契约式编程规定软件设计人员应该为软件组件定义精确的接口规范。

例如，你可以在所有的函数输入中使用断言：

In [2]:
def add(x, y):
    assert isinstance(x, int), 'Expected int'
    assert isinstance(y, int), 'Expected int'
    return x + y

如果函数调用者没有使用正确的参数，那么检查输入可以立即捕捉到。

In [3]:
add(2, 3)

5

In [4]:
add('2','3')

AssertionError: Expected int

#### **内联测试**
断言也可以用于简单的测试。

In [6]:
def add(x, y):
    return x + y
assert add(2, 3) == 5

这样，你就可以将测试与代码包含在同一模块中。

好处：如果代码明显被破坏，那么尝试导入模块将会导致程序崩溃。

对于详尽的测试，不推荐这样做。这种做法更像是基本的“冒烟测试（smoke test）”。函数是否可以在所有的用例上正常工作？如果不可以，那么肯定是有问题的。
#### **`unittest` 模块**
假设你有下面这样一段代码：
```
# simple.py

def add(x, y):
    return x + y
```
现在，你想对这些代码进行测试，请创建一个单独的测试文件，如下所示：
```
# test_simple.py

import simple
import unittest
```
然后定义一个测试类：
```
# test_simple.py

import simple
import unittest

# Notice that it inherits from unittest.TestCase
class TestAdd(unittest.TestCase):
    ...
```
测试类必须继承自`unittest.TestCase`。

在测试类中，定义测试方法：
```
# test_simple.py

import simple
import unittest

# Notice that it inherits from unittest.TestCase
class TestAdd(unittest.TestCase):
    def test_simple(self):
        # Test with simple integer arguments
        r = simple.add(2, 2)
        self.assertEqual(r, 5)
    def test_str(self):
        # Test with strings
        r = simple.add('hello', 'world')
        self.assertEqual(r, 'helloworld')
```
重要提示：每个方法的名称必须以 `test` 开头。

#### **使用 unittest**
`unittest` 中内置了一些断言，每种断言对不同的事情进行诊断。
```
# 验证表达式为真
self.assertTrue(expr)

# 验证 x == y
self.assertEqual(x,y)

# 验证 x != y
self.assertNotEqual(x,y)

# 断言 x 接近 y
self.assertAlmostEqual(x,y,places)

# Assert that callable(arg1,arg2,...) raises exc
self.assertRaises(exc, callable, arg1, arg2, ...)
```
上述列表并不是一个完整的列表，模块还有其它断言。
#### **运行 unittest**
要运行测试，请把代码转换为脚本。
```
# test_simple.py
...

if __name__ == '__main__':
    unittest.main()
```

然后使用 Python 执行测试文件：
```
 bash % python3 test_simple.py
F.

FAIL: test_simple (__main__.TestAdd)

Traceback (most recent call last):
  File "testsimple.py", line 8, in test_simple
    self.assertEqual(r, 5)
AssertionError: 4 != 5

Ran 2 tests in 0.000s
FAILED (failures=1)
```

#### **说明**
高效的单元测试是一种艺术。对于大型应用而言，单元测试可能会变得非常复杂。

`unittest` 模块具有大量与测试运行器（test runners），测试结果集（collection of results）以及测试其他方面相关的选项。相关详细信息，请查阅文档。
#### **第三方测试工具**
虽然内置 `unittest` 模块的优势是可以随处使用——因为它是 Python 的一部分，但是许多程序员也觉得 `unittest` 非常繁琐。另一个流行的的测试工具是 `pytest`。使用 `pytest`，测试文件可以简化为以下形式：
```
# test_simple.py
import simple

def test_simple():
    assert simple.add(2,2) == 4

def test_str():
    assert simple.add('hello','world') == 'helloworld'
```
要运行测试，只需要输入一个命令即可，例如：`python -m pytest` 。它将会发现所有的测试并运行这些测试。

除了这个示例之外，`pytest` 还有很多内容。如果你决定尝试一下，通常很容易上手。

### 练习
在本次练习中，我们将探索使用 Python `unittest` 模块的基本机制（mechanics）。

在前面的练习中，我们编写了一个包含 `Stock` 类的 `stock.py` 文件。对于本次练习，假设我们使用的是 练习7.9 中编写的与类型化属性相关的代码（译注：`typedproperty.py`）。如果因为某些原因，练习 7.9 的代码无法正常工作，你可以从 `Solutions/7_9 `中复制 `typedproperty.py` 到工作目录中。
#### **练习 8.1：编写单元测试**
请创建一个单独的 `test_stock.py` 文件，为 `Stock` 编写单元测试集。为了让你入门，这里有一小段测试实例创建的代码：
```
# test_stock.py

import unittest
import stock

class TestStock(unittest.TestCase):
    def test_create(self):
        s = stock.Stock('GOOG', 100, 490.1)
        self.assertEqual(s.name, 'GOOG')
        self.assertEqual(s.shares, 100)
        self.assertEqual(s.price, 490.1)

if __name__ == '__main__':
    unittest.main()
```
运行单元测试，你应该可以获得一些像下面这样的输出：
.
----------------------------------------------------------------------
Ran 1 tests in 0.000s

OK

然后，编写其它单元测试来检查以下各项内容：
* 确保 `s.cost` 属性返回正确的值（49010.0）。
* 确保 `s.sell()` 方法正常工作。它应该相应地减小 s.shares。
* 确保 `s.shares` 属性只能设置为整数值。

对于最后一部分，你需要检查异常的触发。要做到这些，一种简单的方法是使用如下代码：
```
class TestStock(unittest.TestCase):
    ...
    def test_bad_shares(self):
         s = stock.Stock('GOOG', 100, 490.1)
         with self.assertRaises(TypeError):
             s.shares = '100'
```

### 02 日志
本节对日志模块（logging）进行了介绍。
#### **logging 模块**
`logging` 模块是用于记录诊断信息的Python标准库模块。日志模块非常庞大，具有许多复杂的功能。我们将会展示一个简单的例子来说明其用处。
#### **再探异常**
在本节练习中，我们创建了一个`parse()`函数
```
# fileparse.py

def split(line, types=None, names=None, delimiter=None):
    """分割字符串并应用类型转换"""
    items = line.split(delimiter if delimiter else None)

    if types:
        items = [ty(val) for ty, val in zip(types, items)]

    if names:
        return dict(zip(names, items))

    return items

def parse(f, types=None, names=None, delimiter=None):
    records = []
    for line in f:
        line = line.strip()
        if not line: continue
        try:
            records.append(split(line,types,names,delimiter))
        except ValueError as e:
            print("Couldn't parse :", line)
            print("Reason :", e)
    return records
```
请看到 `try-except` 语句，在 `except` 块中，我们应该做什么？

应该打印警告消息（warning message）？
```
try:
    records.append(split(line,types,names,delimiter))
except ValueError as e:
    print("Couldn't parse :", line)
    print("Reason :", e)
```
还是默默忽略警告消息？
```
try:
    records.append(split(line,types,names,delimiter))
except ValueError as e:
    pass
```
任何一种方式都无法令人满意，通常情况下，两种方式我们都需要（用户可选）。
#### **使用 logging**
`logging` 模块可以解决这个问题：

In [ ]:
# fileparse.py
import logging
log = logging.getLogger(__name__)

def main():
    # 配置日志系统
    logging.basicConfig(
        filename='app.log',        # 日志文件
        level=logging.WARNING,     # 记录级别
        format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    # 示例数据
    data = [
        "1,2,3",
        "a,b,c",    # 这行会触发警告
        "4,5,6"
    ]

    # 调用解析函数
    result = parse(data,
                  types=[int, int, int],
                  names=['x', 'y', 'z'],
                  delimiter=',')

    print("解析结果:", result)

if __name__ == '__main__':
    main()

修改代码以使程序能够遇到问题的时候发出警告消息，或者特殊的 `Logger` 对象。 `Logger` 对象使用 `logging.getLogger(__name__)` 创建。

#### **日志基础**
创建一个记录器对象（Logger Object）。
`log = logging.getLogger(name)  #name是一个字符串`
发出日志消息：
```
log.critical(message [, args])
log.error(message [, args])
log.warning(message [, args])
log.info(message [, args])
log.debug(message [, args])
```
不同方法代表不同级别的严重性。

所有的方法都创建格式化的日志消息。`args`和`%'运算符一起使用以创建消息。
`logmsg = message % args # 写入日志文件`
#### **日志配置**
配置
```
# main.py
...
if __name__ == '__main__':
    import logging
    logging.basicConfig(
        filename='app.log', # 日志文件
        level=logging.Info  # 记录级别
    )
```
通常，在程序启动时，日志配置是一次性的（程序启动后无法重新配置）。该配置与日志调用时分开的。
#### **说明**
日志是可以任意配置的。你可以对日志配置的任何一方面进行调整：如输出文件，级别，消息格式等等，不必担心对使用日志模块的代码造成影响。

### 练习
#### **练习 8.2：将日志添加到模块中**
在`fileparse.py`文件中，有一些与异常有关的错误处理，这些异常时由错误输入引起的。如下所示：
```
# fileparse.py
import csv

def parse_csv(lines, select=None, types=None, has_headers=True, delimiter=',', silence_errors=False):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    if select and not has_headers:
        raise RuntimeError('select requires column headers')

    rows = csv.reader(lines, delimiter=delimiter)

    # Read the file headers (if any)
    headers = next(rows) if has_headers else []

    # If specific columns have been selected, make indices for filtering and set output columns
    if select:
        indices = [ headers.index(colname) for colname in select ]
        headers = select

    records = []
    for rowno, row in enumerate(rows, 1):
        if not row:     # Skip rows with no data
            continue

        # If specific column indices are selected, pick them out
        if select:
            row = [ row[index] for index in indices]

        # Apply type conversion to the row
        if types:
            try:
                row = [func(val) for func, val in zip(types, row)]
            except ValueError as e:
                if not silence_errors:
                    print(f"Row {rowno}: Couldn't convert {row}")
                    print(f"Row {rowno}: Reason {e}")
                continue

        # Make a dictionary or a tuple
        if headers:
            record = dict(zip(headers, row))
        else:
            record = tuple(row)
        records.append(record)

    return records
```
请注意发出诊断消息的 `print` 语句。使用日志操作来替换这些 `print` 语句相对来说更简单。请像下面这样修改代码：
```
# fileparse.py
import csv
import logging
log = logging.getLogger(__name__)

def parse_csv(lines, select=None, types=None, has_headers=True, delimiter=',', silence_errors=False):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    if select and not has_headers:
        raise RuntimeError('select requires column headers')

    rows = csv.reader(lines, delimiter=delimiter)

    # Read the file headers (if any)
    headers = next(rows) if has_headers else []

    # If specific columns have been selected, make indices for filtering and set output columns
    if select:
        indices = [ headers.index(colname) for colname in select ]
        headers = select

    records = []
    for rowno, row in enumerate(rows, 1):
        if not row:     # Skip rows with no data
            continue

        # If specific column indices are selected, pick them out
        if select:
            row = [ row[index] for index in indices]

        # Apply type conversion to the row
        if types:
            try:
                row = [func(val) for func, val in zip(types, row)]
            except ValueError as e:
                if not silence_errors:
                    log.warning("Row %d: Couldn't convert %s", rowno, row)
                    log.debug("Row %d: Reason %s", rowno, e)
                continue

        # Make a dictionary or a tuple
        if headers:
            record = dict(zip(headers, row))
        else:
            record = tuple(row)
        records.append(record)

    return records
```
完成修改后，尝试在错误的数据上使用这些代码：

In [2]:
import report
a = report.read_portfolio('../Data/missing.csv')

第4行: 无法转换 ['MSFT', '', '51.23']
第4行: 原因 invalid literal for int() with base 10: ''
第7行: 无法转换 ['IBM', '', '70.44']
第7行: 原因 invalid literal for int() with base 10: ''


如果你什么都不做，则只会获得 `WARNING` 级别以上的日志消息。输出看起来像简单的打印语句。但是，如果你配置了日志模块，你将会获得有关日志级别，模块等其它信息。请按以下步骤操作查看：

In [3]:
import logging
logging.basicConfig()
a = report.read_portfolio('../Data/missing.csv')

第4行: 无法转换 ['MSFT', '', '51.23']
第4行: 原因 invalid literal for int() with base 10: ''
第7行: 无法转换 ['IBM', '', '70.44']
第7行: 原因 invalid literal for int() with base 10: ''


你会发现，看不到来自于 `log.debug() `操作的输出。请按以下步骤修改日志级别（译注：因为日志配置是一次性的，所以该操作需要重启命令行窗口）：

In [4]:
logging.getLogger('fileparse').level = logging.DEBUG
a = report.read_portfolio('../Data/missing.csv')

第4行: 无法转换 ['MSFT', '', '51.23']
第4行: 原因 invalid literal for int() with base 10: ''
第7行: 无法转换 ['IBM', '', '70.44']
第7行: 原因 invalid literal for int() with base 10: ''


只留下 `critical` 级别的日志消息，关闭其它级别的日志消息。

In [5]:
logging.getLogger('fileparse').level = logging.CRITICAL
a = report.read_portfolio('../Data/missing.csv')

第4行: 无法转换 ['MSFT', '', '51.23']
第4行: 原因 invalid literal for int() with base 10: ''
第7行: 无法转换 ['IBM', '', '70.44']
第7行: 原因 invalid literal for int() with base 10: ''


#### **练习 8.3：向程序添加日志**
要添加日志到应用中，你需要某种机制来实现在主模块中初始化日志。其中一种方式使用看起来像下面这样的代码：
```
# This file sets up basic configuration of the logging module.
# Change settings here to adjust logging output as needed.
import logging
logging.basicConfig(
    filename = 'app.log',            # Name of the log file (omit to use stderr)
    filemode = 'w',                  # File mode (use 'a' to append)
    level    = logging.WARNING,      # Logging level (DEBUG, INFO, WARNING, ERROR, or CRITICAL)
)
```
再次说明，你需要将日志配置代码放到程序启动步骤中。例如，将其放到 report.py 程序里的什么位置？

### 03 调试
#### **调试建议 **
点击`blah.py`文件则可以打开该文件，那么下面的执行请在终端中执行：

假设程序崩溃了：
```
bash % python3 blah.py
Traceback (most recent call last):
  File "blah.py", line 13, in ?
    foo()
  File "blah.py", line 10, in foo
    bar()
  File "blah.py", line 7, in bar
    spam()
  File "blah.py", 4, in spam
    line x.append(3)
AttributeError: 'int' object has no attribute 'append'
```
那么现在该怎么办呢？
#### **阅读回溯信息**
最后一行是程序崩溃的具体原因：
```
bash % python3 blah.py
Traceback (most recent call last):
  File "blah.py", line 13, in ?
    foo()
  File "blah.py", line 10, in foo
    bar()
  File "blah.py", line 7, in bar
    spam()
  File "blah.py", 4, in spam
    line x.append(3)
# Cause of the crash
AttributeError: 'int' object has no attribute 'append'
```
不过，回溯信息并不总是那么易于阅读或理解。

专业建议：将整个回溯粘贴到谷歌。
#### **使用交互式解释器（REPL）**
执行脚本的 时候，可以使用选项 -i 使 Python 保持存活（keep alive）。
```
bash % python3 -i blah.py
Traceback (most recent call last):
  File "blah.py", line 13, in ?
    foo()
  File "blah.py", line 10, in foo
    bar()
  File "blah.py", line 7, in bar
    spam()
  File "blah.py", 4, in spam
    line x.append(3)
AttributeError: 'int' object has no attribute 'append'
>>>
```
选项 -i 可以保留解释器状态。这意味着可以在程序崩溃后查找错误信息。对变量的值和其它状态进行检查。
#### **使用打印进行调试**
使用 `print() `函数进行调试非常常见。

建议：确保使用的是 repr() 函数。
```
def spam(x):
    print('DEBUG:', repr(x))
    ...
```
`repr()` 函数显示一个值的准确表示，而不是格式良好的输出。
```
>>> from decimal import Decimal
>>> x = Decimal('3.4')
# NO `repr`
>>> print(x)
3.4
# WITH `repr`
>>> print(repr(x))
Decimal('3.4')
>>>
```
#### **Python 的调试器**
可以在程序内手动启动调试器（debugger）。
```
def some_function():
    ...
    breakpoint()      # Enter the debugger (Python 3.7+)
    ...
```
上述操作会在 `breakpoint()` 调用时启动调试器。

在 Python 的早期版本中，可能会看到下面这样的调试指南：
```
import pdb
...
pdb.set_trace()       # Instead of `breakpoint()`
...
```
（译注：Python 3.7 之后，可以使用内置函数 breakpoint() 代替 import pdb; pdb.set_trace()）
#### **在调试解释器下运行程序**
也可以在调试器下运行整个程序：
```
bash % python3 -m pdb someprogram.py
```
上述操作会在第一行语句之前自动进入调试器，允许设置断点和修改配置。

常见的调试器命令：
```
(Pdb) help            # Get help
(Pdb) w(here)         # Print stack trace
(Pdb) d(own)          # Move down one stack level
(Pdb) u(p)            # Move up one stack level
(Pdb) b(reak) loc     # Set a breakpoint
(Pdb) s(tep)          # Execute one instruction
(Pdb) c(ontinue)      # Continue execution
(Pdb) l(ist)          # List source code
(Pdb) a(rgs)          # Print args of current function
(Pdb) !statement      # Execute statement
```
断点的位置可以用下列任意一种方式进行表示：
```
(Pdb) b 45            # Line 45 in current file
(Pdb) b file.py:45    # Line 34 in file.py
(Pdb) b foo           # Function foo() in current file
(Pdb) b module.foo    # Function foo() in a module
```
### 练习
#### **练习 8.4：Bugs? 什么是 Bugs?**
有 bug，我们就解决 bug（It runs. Ship it!）。